In [38]:
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np
from PIL import Image


In [39]:
df = pd.read_csv('/Users/user/repos/zoobot-3d/data/gz3d_and_gz_desi_matches.csv')

In [40]:
# df = df[:100]  # debugging mode

In [41]:
#  '/share/nas2/walml/galaxy_zoo/decals/dr8/fits/'
df['galahad_fits_loc'] = df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.fits'


#  '/share/nas2/walml/galaxy_zoo/decals/dr8/jpg/'
df['galahad_jpg_loc'] = df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.jpg'

df['galahad_fits_loc'][0], df['galahad_jpg_loc'][0]

('593252/593252_2670.fits', '593252/593252_2670.jpg')

In [42]:
# df['dr8_id'].value_counts()
# TODO drop duplicate matches, temporarily
df = df.drop_duplicates(subset='dr8_id', keep=False)

In [43]:
with open('data/galahad_fits_to_copy.txt', 'w') as f:
    f.write('\n'.join(df['galahad_fits_loc']))

with open('data/galahad_jpg_to_copy.txt', 'w') as f:
    f.write('\n'.join(df['galahad_jpg_loc']))

In [44]:
# rsync --files-from data/galahad_fits_to_copy.txt -e 'ssh -A -J walml@external.jb.man.ac.uk' walml@galahad.ast.man.ac.uk:/share/nas2/walml/galaxy_zoo/decals/dr8/fits data/gz_desi/fits

In [45]:
# rsync --files-from data/galahad_jpg_to_copy.txt -e 'ssh -A -J walml@external.jb.man.ac.uk' walml@galahad.ast.man.ac.uk:/share/nas2/walml/galaxy_zoo/decals/dr8/jpg data/gz_desi/jpg

In [46]:
df['local_desi_fits_loc'] = 'data/desi/fits/' + df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.fits'
df['local_desi_jpg_loc'] = 'data/desi/jpg/' + df['brickid'].astype(str) + '/' + df['brickid'].astype(str) + '_' + df['objid'].astype(str) + '.jpg'

In [47]:
df['local_json_loc'] = df['local_gz3d_fits_loc'].apply(lambda x: x.replace('/fits_gz/', '/segmaps/marks/').replace('.fits.gz', '.json'))

In [48]:
df['local_spiral_mask_loc'] = df['local_gz3d_fits_loc'].apply(lambda x: x.replace('/fits_gz/', '/segmaps/masks/').replace('.fits.gz', '_spiral.jpg'))
df['local_bar_mask_loc'] = df['local_gz3d_fits_loc'].apply(lambda x: x.replace('/fits_gz/', '/segmaps/masks/').replace('.fits.gz', '_bar.jpg'))

In [49]:
df['local_spiral_mask_loc'][0]

'data/gz3d/segmaps/masks/gz3d_1-379196_127_14728482_spiral.jpg'

In [50]:
len(df)

28983

In [51]:
df = df[df['local_gz3d_fits_loc'].apply(lambda x: os.path.isfile(x))]
len(df)

28983

In [52]:
# add GZ DESI vote predictions

columns = [
    'dr8_id',
    'smooth-or-featured_featured-or-disk_fraction',
    'disk-edge-on_yes_fraction',
    'has-spiral-arms_yes_fraction',
    'spiral-arm-count_2_fraction'
]
gz_desi_preds = pd.read_parquet(
    '/Users/user/repos/desi-predictions/results/final/gz_desi_deep_learning_catalog_advanced.parquet',
    columns=columns
)

df = pd.merge(df, gz_desi_preds, on='dr8_id', validate='one_to_one', how='inner')

In [53]:
# add GZ DESI votes themselves

from zoobot.shared.schemas import decals_all_campaigns_ortho_schema

desi_vote_locs = [
    '/Volumes/beta/galaxy_zoo/decals/dr8/catalogs/training_catalogs/dr12_ortho_v5_labelled_catalog.parquet',
    '/Volumes/beta/galaxy_zoo/decals/dr8/catalogs/training_catalogs/dr5_ortho_v5_labelled_catalog.parquet',
    '/Volumes/beta/galaxy_zoo/decals/dr8/catalogs/training_catalogs/dr8_ortho_v5_labelled_catalog.parquet',
]

columns = [
    'dr8_id',
] + decals_all_campaigns_ortho_schema.label_cols
gz_desi_votes = pd.concat([pd.read_parquet(loc, columns=columns) for loc in desi_vote_locs])

df = pd.merge(df, gz_desi_votes, on='dr8_id', validate='one_to_one', how='left')
# left because some have no votes (about half, 14k of 29k have votes)
df[decals_all_campaigns_ortho_schema.label_cols] = df[decals_all_campaigns_ortho_schema.label_cols].fillna(0)

# TODO could also add GZ2 votes


In [54]:
len(df)

28983

In [55]:
df.to_csv('data/gz3d_and_gz_desi_master_catalog.csv', index=False)